In [1]:
import pandas as pd
import re
import string
import numpy as np

from sklearn.model_selection import train_test_split
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import pickle
from IPython.display import Image

Using TensorFlow backend.


In [2]:
txt = pd.read_table('./pol-eng/pol.txt', names=['eng', 'pol','idx'])


In [3]:
all_eng_words = set()
all_pol_words = set()
length_list_eng = []
length_list_pol = []
txt.shape

(38699, 3)

In [4]:
#we use the lambda keyword to declare an anonymous function, which is why we refer to them as "lambda functions". 
#An anonymous function refers to a function declared with no name.
txt.pol = txt.pol.apply(lambda x: x.lower())
txt.eng = txt.eng.apply(lambda x: x.lower())

In [5]:
txt

,eng,pol,idx
0,go.,idź.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,hi.,cześć.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
2,run!,uciekaj!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
3,run.,biegnij.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
4,run.,uciekaj.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
...,...,...,...
38694,no matter how much you try to convince people ...,"nieważne, jak bardzo usiłujesz przekonać ludzi...",CC-BY 2.0 (France) Attribution: tatoeba.org #9...
38695,a child who is a native speaker usually knows ...,dziecko zwykle wie o swoim języku ojczystym rz...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
38696,since there are usually multiple websites on a...,zwykle jest wiele stron internetowych na każdy...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
38697,"if you want to sound like a native speaker, yo...","jeśli chcesz mówić jak rodzimy użytkownik, mus...",CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [6]:
txt = txt.drop('idx', 1)

In [7]:
txt

,eng,pol
0,go.,idź.
1,hi.,cześć.
2,run!,uciekaj!
3,run.,biegnij.
4,run.,uciekaj.
...,...,...
38694,no matter how much you try to convince people ...,"nieważne, jak bardzo usiłujesz przekonać ludzi..."
38695,a child who is a native speaker usually knows ...,dziecko zwykle wie o swoim języku ojczystym rz...
38696,since there are usually multiple websites on a...,zwykle jest wiele stron internetowych na każdy...
38697,"if you want to sound like a native speaker, yo...","jeśli chcesz mówić jak rodzimy użytkownik, mus..."


In [8]:
#quote removal
txt.eng = txt.eng.apply(lambda x: re.sub("''", '', x))
txt.pol = txt.pol.apply(lambda x: re.sub("''", '', x))
#special char
txt.eng = txt.eng.apply(lambda x: re.sub("[|\^&+\-%*/=!>]", '', x))
txt.pol = txt.pol.apply(lambda x: re.sub("[|\^&+\-%*/=!>]", '', x))

#punctuation
txt.eng = txt.eng.apply(lambda x: re.sub(r'[^\w\s]', "", x))
txt.pol = txt.pol.apply(lambda x: re.sub(r'[^\w\s]', "", x))

#numbers
txt.eng = txt.eng.apply(lambda x: re.sub("[0-9]", '', x))
txt.pol = txt.pol.apply(lambda x: re.sub("[0-9]", '', x))

#extra spacing
txt.eng = txt.eng.replace("/^\s+|\s+$|\s+(?=\s)/g", "")
txt.pol = txt.pol.replace("/^\s+|\s+$|\s+(?=\s)/g", "")

#Start and end tokens ?

txt.pol = txt.pol.apply(lambda x : 'START_' + x + '_END')



In [9]:
txt

,eng,pol
0,go,START_idź_END
1,hi,START_cześć_END
2,run,START_uciekaj_END
3,run,START_biegnij_END
4,run,START_uciekaj_END
...,...,...
38694,no matter how much you try to convince people ...,START_nieważne jak bardzo usiłujesz przekonać ...
38695,a child who is a native speaker usually knows ...,START_dziecko zwykle wie o swoim języku ojczys...
38696,since there are usually multiple websites on a...,START_zwykle jest wiele stron internetowych na...
38697,if you want to sound like a native speaker you...,START_jeśli chcesz mówić jak rodzimy użytkowni...


In [10]:
#Complete Vocab sizes for eng and pol
#compute the vocab sizes and the length of max sequence for both the languages
#create 4 python dicts (2 for each lang) to convert a given token into an integer index and vice versa



In [11]:
for eng in txt.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

In [12]:
for pol in txt.pol:
    for word in pol.split():
        if word not in all_pol_words:
            all_pol_words.add(word)

In [13]:
#finds the maximum length of the target sequence
for l in txt.eng:
    length_list_eng.append(len(l.split(' ')))
    
max_length_src = np.max(length_list_eng)    
    

In [14]:
max_length_src

47

In [15]:
#finds the maximum length of the target sequence
for l in txt.pol:
    length_list_pol.append(len(l.split(' ')))
    
max_length_tar = np.max(length_list_pol)    
    

In [16]:
max_length_tar

38

In [17]:
#Sort dictionaries

source_words = sorted(list(all_eng_words))
target_words = sorted(list(all_pol_words))




In [18]:
# Vocab sizes
#increase num_encoder_token by 1, that is num_encoder_token -. vocab_size + 1. 

num_encoder_tokens = len(all_eng_words)+1
num_decoder_tokens = len(all_pol_words)
num_decoder_tokens += 1 #padding
num_encoder_tokens, num_decoder_tokens

(8201, 28427)

In [19]:
#WORD TO TOKEN DICTIONARY

source_token_index = dict([(word, i+1) for i, word in enumerate(source_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [20]:
source_token_index

{'a': 1,
 'abandon': 2,
 'abandoned': 3,
 'abducted': 4,
 'abhor': 5,
 'abilities': 6,
 'ability': 7,
 'able': 8,
 'aboard': 9,
 'abolish': 10,
 'abolished': 11,
 'about': 12,
 'above': 13,
 'abraham': 14,
 'abreast': 15,
 'abroad': 16,
 'abrupt': 17,
 'absence': 18,
 'absent': 19,
 'absolutely': 20,
 'absorb': 21,
 'absorbed': 22,
 'absorbs': 23,
 'abstain': 24,
 'abstract': 25,
 'absurd': 26,
 'abundant': 27,
 'abused': 28,
 'abuses': 29,
 'academic': 30,
 'accelerated': 31,
 'accent': 32,
 'accept': 33,
 'accepted': 34,
 'access': 35,
 'accessible': 36,
 'accident': 37,
 'accidental': 38,
 'accidentally': 39,
 'accidents': 40,
 'acclimated': 41,
 'accompanied': 42,
 'accompany': 43,
 'accomplish': 44,
 'accomplished': 45,
 'accomplishes': 46,
 'according': 47,
 'accordion': 48,
 'account': 49,
 'accountable': 50,
 'accountant': 51,
 'accounted': 52,
 'accounting': 53,
 'accounts': 54,
 'accumulated': 55,
 'accuracy': 56,
 'accurate': 57,
 'accusations': 58,
 'accused': 59,
 'accusin

In [21]:
# Create token to word dicitonary for both source and target

reverse_source_char_index = dict((i,word) for word, i in source_token_index.items())
reverse_target_char_index = dict((i,word) for word, i in target_token_index.items())

In [22]:
#num_decoder_tokens = num_decoder_tokens*1.05
num_decoder_tokens

28427

In [23]:
# Train - Test Split
X, y = txt.eng, txt.pol
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((34829,), (3870,))

In [24]:

#Save the train and test dataframes for reproducing the results later, as they are shuffled.
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [25]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            #matrix of zeros 128*47
            encoder_source_data = np.zeros((batch_size, max_length_src),dtype='float32')
            #matrix of zeros 128*38
            decoder_source_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            #matrix of zeros 128*38*28427
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            
            
            for i, (source_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                #Enumerate() method adds a counter to an iterable and returns it in a form of enumerate object.
                for t, word in enumerate(source_text.split()):
                    
                    encoder_source_data[i, t] = source_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_source_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_source_data, decoder_source_data], decoder_target_data)


In [26]:
#Encoder Decoder Model Architure

latent_dim = 100

In [27]:
#Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [28]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [29]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [30]:
#Image(retina=True, filename='train_model.png')

In [31]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 100
epochs = 50

train_samples, val_samples

(34829, 3870)

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/50
348/348 [==============================] - 1506s 4s/step - loss: 0.8418 - acc: 0.0321 - val_loss: 0.8555 - val_acc: 0.0428
Epoch 2/50
348/348 [==============================] - 2370s 7s/step - loss: 0.7846 - acc: 0.0530 - val_loss: 0.8849 - val_acc: 0.0649
Epoch 3/50
322/348 [==========================>...] - ETA: 2:06 - loss: 0.7501 - acc: 0.0796

In [ ]:
model.save_weights('nmt_weights.h5')q
model.load_weights('nmt_weights.h5')


In [ ]:
#INFERENCE SET UP
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Polish Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Polish Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Polish Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Polish Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Polish Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Polish Translation:', decoded_sentence[:-4])

In [ ]:
#How to take user input??
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)

print('Input English sentence:', 'My name is Paulina')
print('Actual Polish Translation:', 'Mam na imie Paulina')
print('Predicted Polish Translation:', decoded_sentence[:-4])

